### Summarize tidy tables
This script summarizes the water use and water suppy tidy tables generated by the *CreateUsageTable* and *CreateSupplyTable* scripts, respectively. Each table is then merged into a single dataframe to create a table listing water use and supply for each year/state combination. 

In [3]:
#Import libraries
import sys, os
import pandas as pd
import numpy as np

In [10]:
#Get file names
dataDir = '../../Data'
tidyuseFN = dataDir + os.sep + "UsageDataTidy.csv"
tidysupplyFN = dataDir + os.sep + "SupplyTableTidy.csv"

In [45]:
#Read in tidy tables
dfUse = pd.read_csv(tidyuseFN,dtype={'FIPS':np.str})
dfSupply = pd.read_csv(tidysupplyFN,dtype={'FIPS':np.str,'STATEFIPS':np.str})                    

#### Summarize USE table

In [95]:
dfUseFIPS = dfUse.pivot_table(index=['YEAR','FIPS'],aggfunc='sum',columns=['UseClass','SrcClass'])
dfUseFIPS.head()

Amount                                                  \
UseClass   Aquaculture            Domestic          Industrial           
SrcClass   Groundwater Surface Groundwater Surface Groundwater Surface   
YEAR FIPS                                                                
2000 01001         0.0     0.0        2.95     0.0        1.69    26.6   
     01003         0.0     0.0        2.46     0.0        0.79     5.7   
     01005         0.0     0.0        0.51     0.0        0.94     0.0   
     01007         0.0     0.0        0.22     0.0        0.00     0.0   
     01009         0.0     0.0        0.38     0.0        0.00     0.0   

                                                                        \
UseClass    Irrigation           Livestock              Mining           
SrcClass   Groundwater Surface Groundwater Surface Groundwater Surface   
YEAR FIPS                                                                
2000 01001        0.31    0.00         NaN     NaN         NaN     NaN   
     01003        4.09    1.89         NaN     NaN         NaN     NaN   
     01005        0.06    1.06         NaN     NaN         NaN     NaN   
     01007        0.00    0.00         NaN     NaN         NaN     NaN   
     01009        0.00    0.07         NaN     NaN         NaN     NaN   

                                                                         
UseClass   PublicSupply          ThermoElec               Total          
SrcClass    Groundwater Surface Groundwater Surface Groundwater Surface  
YEAR FIPS                                                                
2000 01001         5.70    0.00         0.0     0.0       10.65   26.60  
     01003        19.70    0.00         0.0     0.0       27.04    7.59  
     01005         4.85    0.00         0.0     0.0        6.36    1.06  
     01007         3.78    0.00         0.0     0.0        4.00    0.00  
     01009         2.55   29.26         0.0     0.0        2.93   29.33

In [46]:
#Add STATEFIPS column to dfUse (as left most 2 characters of FIPS values)
dfUse['STATEFIPS'] = dfUse['FIPS'].str[:2]

In [47]:
#Remove irrigation and thermoelectric sub-classes
dropValues = ['Irrigation_Crop', 'Irrigation_Golf','ThermoElec_OnceThru', 'ThermoElec_Recirc']
dfUse = dfUse[~dfUse['UseClass'].isin(dropValues)]

In [48]:
#CHECK - use data frame
dfUse.head(2)

,STATE,FIPS,YEAR,Class,Amount,UseClass,SrcClass,SrcType,STATEFIPS
0,AL,01001,2000,PS-WGWFr,5.7,PublicSupply,Groundwater,Fresh,01
1,AL,01003,2000,PS-WGWFr,19.7,PublicSupply,Groundwater,Fresh,01


In [15]:
#CHECK - supply data frame
dfSupply.head(2)

,LAT,YEAR,LON,total_runoff,et,pr,smc,FIPS,NAME,STATEFIPS,Area
0,48.0625,2000,-124.6875,2181.373291,738.717041,2823.416016,13044.400391,53009,Clallam,53,6.917031e+09
1,48.1875,2000,-124.6875,2167.416016,745.596985,2814.455078,12944.500000,53009,Clallam,53,6.917031e+09


In [82]:
#Compute supply as precipitation - evapotranspiration 
#(See https://www.fs.fed.us/rm/value/docs/spatial_distribution_water_supply.pdf)
# * Could also use total_runoff
# * Values are in mm/year and need to be adjusted to MGal/year by mulitplying by weighted area
dfSupply['Supply_mm'] = dfSupply['pr'] - dfSupply['et']

In [90]:
#Summarize supply on YEAR and FIPS
'''We take the mean mm/year across points in a county and then 
   mulitply by county area to get volume (mm * m3). These values
   then need to by converted to MGal to give MGal/year
'''

#Compute mean runoff and supply on year and county
dfSupplyFIPS = dfSupply.groupby(('YEAR','STATEFIPS','FIPS','Area'))['total_runoff','Supply_mm'].mean()

#Reset the index so Year, StateFIPS, FIPS, and AREA become columns again
dfSupplyFIPS.reset_index(inplace=True)

#Convert mm/Year * county area (m2) into MGal/year - to match use values
''' m = [mm] / 1000; 
    m * [m2] = m3;
   [m3] / 3785.41178 = 1 MGal'''
for param in ('total_runoff','Supply_mm'):
    dfSupplyFIPS[param] = (dfSupplyFIPS[param] / 1000.0) * dfSupplyFIPS.Area / 3785.41178

In [91]:
dfSupplyFIPS.head(2)

,YEAR,STATEFIPS,FIPS,Area,total_runoff,Supply_mm
0,2000,01,01001,1.565359e+09,164183.196753,136727.155553
1,2000,01,01003,5.250715e+09,747212.646056,697763.897642


### Join Use and Suppy Tables on Year and FIPS

In [92]:
dfAll = pd.merge(dfUse,dfSupplyFIPS, how='outer',on=['YEAR','FIPS'])

In [93]:
dfAll.head()

,STATE,FIPS,YEAR,Class,Amount,UseClass,SrcClass,SrcType,STATEFIPS_x,STATEFIPS_y,Area,total_runoff,Supply_mm
0,AL,01001,2000.0,PS-WGWFr,5.70,PublicSupply,Groundwater,Fresh,01,01,1.565359e+09,164183.196753,136727.155553
1,AL,01001,2000.0,PS-WSWFr,0.00,PublicSupply,Surface,Fresh,01,01,1.565359e+09,164183.196753,136727.155553
2,AL,01001,2000.0,DO-WGWFr,2.95,Domestic,Groundwater,Fresh,01,01,1.565359e+09,164183.196753,136727.155553
3,AL,01001,2000.0,DO-WSWFr,0.00,Domestic,Surface,Fresh,01,01,1.565359e+09,164183.196753,136727.155553
4,AL,01001,2000.0,IN-WGWFr,1.69,Industrial,Groundwater,Fresh,01,01,1.565359e+09,164183.196753,136727.155553


In [81]:
#Summarize use data by year, state, and source type (surface vs groundwater)
dfStateUse = dfUse.pivot_table(index=['YEAR','SrcClass'],aggfunc='sum',columns=['UseClass'])
dfSt